# The PolInSAR course - October 4, 2021 
# Synthetic Aperture Radar (SAR) 
# Part 2: Azimuth Resolution

In [1]:
# --- To get started, we set the ipympl backend, which makes matplotlib plots interactive.
# --- We do this using a magic command, starting with %.
%matplotlib widget

# --- import libraries
import sys
sys.path.append('/projects/src/')

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq, fftshift, ifft

# --- import rat class
from ste_io import *

# --- Useful functions

def rect(t, T0):
    # Implements y = rect(t/T0) = 1 if abs(t/T0) <= 0.5, 0 otherwise 
    # All input / outputs are Numpy arrays
    
    N = t.shape[0]
    y = np.zeros(N, 'float32') 
    y[np.abs(t/T0) <= 0.5] = 1
    
    return y

# --- Constants

# speed of light
c0 = 2.9972190e+08


/projects/src/ste_io.py:69: UserWarning: Failed to import STEtools.visualisation. write_pixmap will not work.
  warnings.warn("Failed to import STEtools.visualisation. write_pixmap will not work.")


## Exercise 1

A typical P-band acquisition of DLR's F-SAR system is characterized by the following parameters:

In [3]:
# --- Input parameters 

# SAR wavelength meters
lambda0 = 0.68901587
# platform velocity, meters/second
vp    = 102.89772    
# range at closest point
R0    = 6445.2197      
# pixel spacing in meters
azsp  = 0.90879266    
# number of pixels 
Nsaz  = 9984

The (unfocused) azimuth signal in */projects/data/01-sar/signal1_ac.npy* refers to 1 point-like scatterer. 
Which is the correct value for the azimuth resolution among 0.5 m, 1.5 m, 5 m ?

### Solution

In [11]:
# --- calculate time axis
time_a = np.linspace(0, Nsaz-1, Nsaz) * azsp / vp
time_a = time_a - np.max(time_a) / 2

# --- Calculate Tsa (illumination time)
# azimuth resolution
delta_sa = 1.5
# real antenna aperture
da = 2 * delta_sa
# angular aperture
theta = lambda0 / da
# synthetic aperture
Lsa = theta * R0
# illumination time
Tsa = Lsa / vp

# --- Calculate azimuth chirp rate
ka = 2 * vp**2 / lambda0 / R0

# --- chirp signal
s0_t = np.exp(1j * np.pi * ka * time_a**2) * rect(time_a, Tsa)

# --- matched filter
h0_t = np.conj(s0_t[::-1])
# plot matched filter response
plt.figure()
plt.plot(time_a*vp, np.real(h0_t))
#plt.xlabel('Time (s)')
plt.xlabel('Azimuth distance (m)')
plt.ylabel('Real part')
plt.title('Matched filter response')
plt.grid()

# --- focus input data 1
s1_t = np.load('/projects/data/01-sar/signal1_ac.npy')
u1_t = np.convolve(s1_t, h0_t, 'same')

# plot input signal
plt.figure()
plt.plot(time_a*vp, np.real(s1_t))
plt.xlabel('Azimuth distance (m)')
plt.ylabel('Real part')
plt.title('Input signal 1')
plt.grid()

# plot input signal
plt.figure()
plt.plot(time_a*vp, np.real(u1_t))
plt.xlabel('Azimuth distance (m)')
plt.ylabel('Real part')
plt.title('Output signal 1')
plt.grid()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Exercise 2

Now load the azimuth signal in */projects/data/01-sar/signal2_ac.npy*, and focus it with the azimuth chirp calculated with the resolution determined in Exercise 1. How many scatterer do you see, and where?

### Solution

In [14]:
# --- open signal 2
s2_t = np.load('/projects/data/01-sar/signal2_ac.npy')

# plot input signal
plt.figure()
plt.plot(time_a*vp, np.real(s2_t))
plt.xlabel('Azimuth distance (m)')
plt.ylabel('Real part')
plt.title('Input signal 2')
plt.grid()

# focusing
u2_t = np.convolve(s2_t, h0_t, 'same')

# plot output signal
plt.figure()
plt.plot(time_a*vp, np.real(u2_t))
plt.xlabel('Azimuth distance (m)')
plt.ylabel('Real part')
plt.title('Output signal 2')
plt.grid()


<ipython-input-14-d255368290bb>:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Exercise 3

We are now ready now to focus in both rangea nd azimuth the real F-SAR P-band data already used last week !

1. Open the raw data image in the file */projects/data/01-sar/raw-img.rat*, and visualize it (only the amplitude).
2. Compress it in range using the chirp parameters provided below using the FFT. What do you see?
3. After range focusing, compress the result in azimuth, again using the FFT. What happens if the azimuth chirp is adapted in near range? what happens if the chirp is adapted in far range?

Tip:
Use the provided rat class to open the data - Example: img = rrat(*filename.rat*)

In [15]:

# --- Inputs

# SAR wavelength
lambda0 = 0.68901587

# --- parameters for range focusing

# range chirp duration, s
T   = 1.0016000e-05
# chirp bandwidth, Hz
Wr  = 50.0e6
# range sampling, m
rsp = 1.1988876
# number of samples 
Nsr = 3772

# --- parameters for azimuth focusing

# azimut resolution, m
deltasa = 2/1.33 # 1.5
# platform velocity, meters/second
vp      = 102.89772    
# pixel spacing in meters
azsp    = 0.90879266    
# number of pixels 
Nsaz    = 9984
# range delay, s <---- NEW !!!
rd  = 4.3008000e-05
# the range delay is the amount of time between the chirp transmission
# and the receipt of the first backscattered signal ==> converted in meters it provides the minimum range


### Solution

In [16]:
img = rrat('/projects/data/01-sar/raw-img.rat')
img.shape

(9984, 3772)

In [18]:
# --- generate range chirp
time_r = np.linspace(0, Nsr-1, Nsr) * rsp / (c0/2)
time_r = time_r - np.max(time_r) / 2
kr = Wr / T
sr_t = np.exp(1j * np.pi * kr * time_r**2) * rect(time_r, T)

# --- range focusing
hr_t = np.conj(sr_t[::-1])
img_r = np.zeros(img.shape, 'complex64')

for nn in range(Nsaz) :
    
    u_fft = fft(img[nn, :]) * fft(hr_t)
    img_r[nn, :] = fftshift( ifft(u_fft) )
    
# --- plot before range focusing
plt.figure(figsize = (5, 8))
plt.imshow(np.flipud(abs(img)), vmin = 0, vmax = 2*np.mean(abs(img)), cmap = 'gray')
plt.xlabel('Range (pixels)')
plt.ylabel('Azimuth (pixels)')
plt.title('Raw data')

# --- plot after range focusing
plt.figure(figsize = (5, 8))
plt.imshow(np.flipud(abs(img_r)), vmin = 0, vmax = 2*np.mean(abs(img_r)), cmap = 'gray')
plt.xlabel('Range (pixels)')
plt.ylabel('Azimuth (pixels)')
plt.title('After range focusing')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'After range focusing')

In [19]:
# --- focus in azimuth

# calculate time axis
time_a = np.linspace(0, Nsaz-1, Nsaz) * azsp / vp
time_a = time_a - np.max(time_a) / 2

# real antenna aperture
da = 2 * deltasa
# angular aperture
theta = lambda0 / da
# synthetic aperture
R0 = rd * c0 /2  # first pixel = near range
Lsa = theta * R0
# illumination time
Tsa = Lsa / vp

# --- Calculate azimuth chirp rate
ka = 2 * vp**2 / lambda0 / R0

# --- chirp signal
sa_t = np.exp(1j * np.pi * ka * time_a**2) * rect(time_a, Tsa)

# --- matched filter
ha_t = np.conj(sa_t[::-1])

# --- azimuth focusing
img_ra = np.zeros(img.shape, 'complex64')

for nn in range(Nsr) :
    
    u_fft = fft(img_r[:, nn]) * fft(ha_t)
    img_ra[:, nn] = fftshift( ifft(u_fft) )
    
# --- plot after azimuth focusing
plt.figure(figsize = (5, 8))
plt.imshow(np.flipud(abs(img_ra)), vmin = 0, vmax = 2*np.mean(abs(img_ra)), cmap = 'gray')
plt.xlabel('Range (pixels)')
plt.ylabel('Azimuth (pixels)')
plt.title('After range and azimuth focusing')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'After range and azimuth focusing')